# Word Sentiment Analyser

In [695]:
!pip3 install xgboost
import string 

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to C:\Users\Neha
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Neha
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Neha
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [696]:
#dataset for no text processing

dataset = pd.read_csv(r"C:\Users\Neha Sharma\Desktop\Data driven SE\Assignment-2\Lab 2_icsme-questions-labeled.csv");
dataset.head()

,inline-comment-id,# Comment,Question,Low-level Label,Top-level Label
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,request for confirmation,request
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,surprise,attitudes and emotions
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,suggestion,suggestion
3,193d089f_f5fac752,4,i can't see anywhere where this is set to fals...,suggestion,suggestion
4,50c2f81e_ac4fd6fc,5,are you sure you want to include this source f...,criticism,attitudes and emotions


## Dataset Preprocessing

In [697]:
#dataset for preprocessing

dataset_preprocessed = pd.read_csv(r"C:\Users\Neha Sharma\Desktop\Data driven SE\Assignment-2\Lab 2_icsme-questions-labeled.csv");
dataset_preprocessed.head()

,inline-comment-id,# Comment,Question,Low-level Label,Top-level Label
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,request for confirmation,request
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,surprise,attitudes and emotions
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,suggestion,suggestion
3,193d089f_f5fac752,4,i can't see anywhere where this is set to fals...,suggestion,suggestion
4,50c2f81e_ac4fd6fc,5,are you sure you want to include this source f...,criticism,attitudes and emotions


In [698]:
#Covert to String
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].astype(str)

#Covert all characters to lower
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].str.lower()

#Remove punchuation marks
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

#Remove Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

#Removing Frequent words
from collections import Counter
cnt = Counter()
for text in dataset_preprocessed["Question"].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(10)

FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

#Remove Rare Words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

#stemming
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

#Lemmetizing
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


#calling all functions
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_punctuation(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_stopwords(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_punctuation(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_stopwords(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_freqwords(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: remove_rarewords(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: stem_words(text))
dataset_preprocessed["Question"] = dataset_preprocessed["Question"].apply(lambda text: lemmatize_words(text))

#dataset after preprocessing
dataset_preprocessed.head()

,inline-comment-id,# Comment,Question,Low-level Label,Top-level Label
0,84326dd1_566c7146,1,intend dont realli want targetoutdatanativetest,request for confirmation,request
1,84326dd1_566c7146,2,intend dont realli want targetoutdatanativetest,surprise,attitudes and emotions
2,99d1f8e4_92b31cea,3,dont need increment el case avoid infinit loop,suggestion,suggestion
3,193d089f_f5fac752,4,cant see anywher set fals adjust singl refer b...,suggestion,suggestion
4,50c2f81e_ac4fd6fc,5,sure want includ sourc file directli creat sta...,criticism,attitudes and emotions


## Random forest - after preprocessing

In [699]:
#X_p <- Feature variables set with data pre-processing
#y_p <- Target variables set with data pre-processing

#X_train_p,y_train_p <- Training Sets with processed data
#X_test_p, y_test_p <- Test Sets with processed data

X_p = dataset_preprocessed.iloc[:,2]
y_p = dataset_preprocessed.iloc[:,4].values

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p,test_size=0.1, random_state=42)
print(X_train_p.shape, X_test_p.shape, y_train_p.shape, y_test_p.shape)

(444,) (50,) (444,) (50,)


In [700]:
#Using Count Vectorizer
vectorizer_p = CountVectorizer()

In [701]:
print(y_train_p.shape)
print(y_test_p.shape)

(444,)
(50,)


In [702]:
X_train_p = vectorizer_p.fit_transform(X_train_p)
X_test_p = vectorizer_p.transform(X_test_p)

In [703]:
print(X_train_p.shape)
print(y_train_p.shape)
print(X_test_p.shape)
print(y_test_p.shape)

(444, 1484)
(444,)
(50, 1484)
(50,)


In [704]:
#Random forest model implementation
rf_p = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf_p.fit(X_train_p, y_train_p)
print("Training set score :{}".format(rf_p.score(X_train_p,y_train_p)))
print("Test set score :{}".format(rf_p.score(X_test_p,y_test_p)))

Training set score :0.8738738738738738
Test set score :0.46


In [705]:
#confusion matrix
y_pred_p = rf_p.predict(X_test_p)
cm_p = confusion_matrix(y_test_p, y_pred_p)
print(cm_p)

[[ 0  0  4  0  1]
 [ 0  0  1  0  0]
 [ 1  2 17  1  9]
 [ 0  0  0  0  0]
 [ 0  0  8  0  6]]


In [706]:
#classification Report
print(classification_report(y_test_p, y_pred_p))

                        precision    recall  f1-score   support

attitudes and emotions       0.00      0.00      0.00         5
 hypothetical scenario       0.00      0.00      0.00         1
               request       0.57      0.57      0.57        30
   rhetorical question       0.00      0.00      0.00         0
            suggestion       0.38      0.43      0.40        14

              accuracy                           0.46        50
             macro avg       0.19      0.20      0.19        50
          weighted avg       0.45      0.46      0.45        50



In [707]:
#finding accuracy
from sklearn.model_selection import cross_validate
avg_acc_score=0
for i, score in enumerate(cross_validate(rf_p, X_train_p, y_train_p, scoring='accuracy', cv=10)["test_score"]):
    avg_acc_score += score
    print(f"Accuracy for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Accuracy is: {avg_acc_score/10}")
print(f"\naverage 10-fold cross validation Accuracy percentage is: {(avg_acc_score/10)*100}")

Accuracy for the fold no. 0 on the test set: 0.6
Accuracy for the fold no. 1 on the test set: 0.4888888888888889
Accuracy for the fold no. 2 on the test set: 0.4888888888888889
Accuracy for the fold no. 3 on the test set: 0.4888888888888889
Accuracy for the fold no. 4 on the test set: 0.4772727272727273
Accuracy for the fold no. 5 on the test set: 0.5681818181818182
Accuracy for the fold no. 6 on the test set: 0.5227272727272727
Accuracy for the fold no. 7 on the test set: 0.5227272727272727
Accuracy for the fold no. 8 on the test set: 0.5227272727272727
Accuracy for the fold no. 9 on the test set: 0.5227272727272727

average 10-fold cross validation Accuracy is: 0.5203030303030303

average 10-fold cross validation Accuracy percentage is: 52.030303030303024


In [708]:
#finding precision
avg_precision=0
for i, score in enumerate(cross_validate(rf_p, X_train_p, y_train_p, scoring='precision_weighted', cv=10)["test_score"]):
    avg_precision += score
    print(f"Precision for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Precision is: {avg_precision/10}")
print(f"\naverage 10-fold cross validation Precision percentage is: {(avg_precision/10)*100}")

Precision for the fold no. 0 on the test set: 0.5674603174603174
Precision for the fold no. 1 on the test set: 0.42857142857142855
Precision for the fold no. 2 on the test set: 0.43606701940035275
Precision for the fold no. 3 on the test set: 0.397979797979798
Precision for the fold no. 4 on the test set: 0.42424242424242425
Precision for the fold no. 5 on the test set: 0.5010822510822511
Precision for the fold no. 6 on the test set: 0.465564738292011
Precision for the fold no. 7 on the test set: 0.46529484029484036
Precision for the fold no. 8 on the test set: 0.43087121212121215
Precision for the fold no. 9 on the test set: 0.4693181818181818

average 10-fold cross validation Precision is: 0.4586452211262818

average 10-fold cross validation Precision percentage is: 45.86452211262818


In [709]:
#finding recall
avg_recall=0
for i, score in enumerate(cross_validate(rf_p, X_train_p, y_train_p, scoring='recall_weighted', cv=10)["test_score"]):
    avg_recall += score
    print(f"Recall for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Recall is: {avg_recall/10}")
print(f"\naverage 10-fold cross validation Recall percentage is: {(avg_recall/10)*100} ")

Recall for the fold no. 0 on the test set: 0.6
Recall for the fold no. 1 on the test set: 0.4888888888888889
Recall for the fold no. 2 on the test set: 0.4888888888888889
Recall for the fold no. 3 on the test set: 0.4888888888888889
Recall for the fold no. 4 on the test set: 0.4772727272727273
Recall for the fold no. 5 on the test set: 0.5681818181818182
Recall for the fold no. 6 on the test set: 0.5227272727272727
Recall for the fold no. 7 on the test set: 0.5227272727272727
Recall for the fold no. 8 on the test set: 0.5227272727272727
Recall for the fold no. 9 on the test set: 0.5227272727272727

average 10-fold cross validation Recall is: 0.5203030303030303

average 10-fold cross validation Recall percentage is: 52.030303030303024 


In [710]:
#finding f1 score
from sklearn.model_selection import cross_validate
f1_score=0
for i, score in enumerate(cross_validate(rf_p, X_train_p, y_train_p, scoring='f1_weighted', cv=10)["test_score"]):
    f1_score += score
    print(f"f1-score for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation f1-score is: {f1_score/10}")
print(f"\naverage 10-fold cross validation f1-score percentage is: {(f1_score/10)*100}")

f1-score for the fold no. 0 on the test set: 0.5822308971300514
f1-score for the fold no. 1 on the test set: 0.43386243386243384
f1-score for the fold no. 2 on the test set: 0.43699287961583044
f1-score for the fold no. 3 on the test set: 0.43247863247863244
f1-score for the fold no. 4 on the test set: 0.41363636363636364
f1-score for the fold no. 5 on the test set: 0.5118451463790447
f1-score for the fold no. 6 on the test set: 0.4772727272727273
f1-score for the fold no. 7 on the test set: 0.4550777091760699
f1-score for the fold no. 8 on the test set: 0.45256916996047436
f1-score for the fold no. 9 on the test set: 0.48989898989898983

average 10-fold cross validation f1-score is: 0.4685864949410618

average 10-fold cross validation f1-score percentage is: 46.85864949410618


##  Improved model (Kernel SVM) - with preprocessing

In [713]:
#SVM model
svm_kernel_p = SVC(kernel = 'rbf', random_state = 0)
svm_kernel_p.fit(X_train_p, y_train_p)

y_pred_ksvm_p = svm_kernel_p.predict(X_test_p)
cm_p = confusion_matrix(y_test_p, y_pred_ksvm_p)
print(cm_p)
print(accuracy_score(y_test_p, y_pred_ksvm_p))
print(classification_report(y_test_p, y_pred_ksvm_p))
print("Training set score :{}".format(svm_kernel_p.score(X_train_p,y_train_p)))
print("Test set score :{}".format(svm_kernel_p.score(X_test_p,y_test_p)))

[[ 0  0  5  0  0]
 [ 0  0  1  0  0]
 [ 0  2 25  1  2]
 [ 0  0  0  0  0]
 [ 0  0 10  0  4]]
0.58
                        precision    recall  f1-score   support

attitudes and emotions       0.00      0.00      0.00         5
 hypothetical scenario       0.00      0.00      0.00         1
               request       0.61      0.83      0.70        30
   rhetorical question       0.00      0.00      0.00         0
            suggestion       0.67      0.29      0.40        14

              accuracy                           0.58        50
             macro avg       0.26      0.22      0.22        50
          weighted avg       0.55      0.58      0.53        50

Training set score :0.759009009009009
Test set score :0.58


In [714]:
from sklearn.model_selection import cross_validate
avg_acc_score=0
for i, score in enumerate(cross_validate(svm_kernel_p, X_train_p, y_train_p, scoring='accuracy', cv=10)["test_score"]):
    avg_acc_score += score
    print(f"Accuracy for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Accuracy is: {avg_acc_score/10}")
print(f"\naverage 10-fold cross validation Accuracy percentage is: {(avg_acc_score/10)*100}")

Accuracy for the fold no. 0 on the test set: 0.5777777777777777
Accuracy for the fold no. 1 on the test set: 0.5111111111111111
Accuracy for the fold no. 2 on the test set: 0.4888888888888889
Accuracy for the fold no. 3 on the test set: 0.4888888888888889
Accuracy for the fold no. 4 on the test set: 0.5
Accuracy for the fold no. 5 on the test set: 0.5454545454545454
Accuracy for the fold no. 6 on the test set: 0.5227272727272727
Accuracy for the fold no. 7 on the test set: 0.5681818181818182
Accuracy for the fold no. 8 on the test set: 0.5681818181818182
Accuracy for the fold no. 9 on the test set: 0.5454545454545454

average 10-fold cross validation Accuracy is: 0.5316666666666666

average 10-fold cross validation Accuracy percentage is: 53.166666666666664


In [715]:
avg_precision=0
for i, score in enumerate(cross_validate(svm_kernel_p, X_train_p, y_train_p, scoring='precision_weighted', cv=10)["test_score"]):
    avg_precision += score
    print(f"Precision for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Precision is: {avg_precision/10}")
print(f"\naverage 10-fold cross validation Precision percentage is: {(avg_precision/10)*100}")

Precision for the fold no. 0 on the test set: 0.49430199430199434
Precision for the fold no. 1 on the test set: 0.4181286549707603
Precision for the fold no. 2 on the test set: 0.4035087719298245
Precision for the fold no. 3 on the test set: 0.2861788617886179
Precision for the fold no. 4 on the test set: 0.4007177033492823
Precision for the fold no. 5 on the test set: 0.4718899521531101
Precision for the fold no. 6 on the test set: 0.43779904306220097
Precision for the fold no. 7 on the test set: 0.533259423503326
Precision for the fold no. 8 on the test set: 0.6453488372093023
Precision for the fold no. 9 on the test set: 0.4521531100478469

average 10-fold cross validation Precision is: 0.4543286352316266

average 10-fold cross validation Precision percentage is: 45.43286352316266


In [716]:
avg_recall=0
for i, score in enumerate(cross_validate(svm_kernel_p, X_train_p, y_train_p, scoring='recall_weighted', cv=10)["test_score"]):
    avg_recall += score
    print(f"Recall for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Recall is: {avg_recall/10}")
print(f"\naverage 10-fold cross validation Recall percentage is: {(avg_recall/10)*100} ")

Recall for the fold no. 0 on the test set: 0.5777777777777777
Recall for the fold no. 1 on the test set: 0.5111111111111111
Recall for the fold no. 2 on the test set: 0.4888888888888889
Recall for the fold no. 3 on the test set: 0.4888888888888889
Recall for the fold no. 4 on the test set: 0.5
Recall for the fold no. 5 on the test set: 0.5454545454545454
Recall for the fold no. 6 on the test set: 0.5227272727272727
Recall for the fold no. 7 on the test set: 0.5681818181818182
Recall for the fold no. 8 on the test set: 0.5681818181818182
Recall for the fold no. 9 on the test set: 0.5454545454545454

average 10-fold cross validation Recall is: 0.5316666666666666

average 10-fold cross validation Recall percentage is: 53.166666666666664 


In [717]:
#finding f1 score
from sklearn.model_selection import cross_validate
f1_score=0
for i, score in enumerate(cross_validate(svm_kernel_p, X_train_p, y_train_p, scoring='f1_weighted', cv=10)["test_score"]):
    f1_score += score
    print(f"f1-score for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation f1-score is: {f1_score/10}")
print(f"\naverage 10-fold cross validation f1-score percentage is: {(f1_score/10)*100}")

f1-score for the fold no. 0 on the test set: 0.49454365079365076
f1-score for the fold no. 1 on the test set: 0.4338624338624339
f1-score for the fold no. 2 on the test set: 0.4162257495590828
f1-score for the fold no. 3 on the test set: 0.361025641025641
f1-score for the fold no. 4 on the test set: 0.41684122329283624
f1-score for the fold no. 5 on the test set: 0.4669040636782572
f1-score for the fold no. 6 on the test set: 0.437683284457478
f1-score for the fold no. 7 on the test set: 0.46177156177156176
f1-score for the fold no. 8 on the test set: 0.4333870420624152
f1-score for the fold no. 9 on the test set: 0.45527859237536655

average 10-fold cross validation f1-score is: 0.43775232428787236

average 10-fold cross validation f1-score percentage is: 43.775232428787234


# Random forest - without processing 

In [718]:
#X <- Feature variables set without data pre-processing
#y <- Target variables set without data pre-processing

#X_train,y_train <- Training Sets without processed data
#X_test, y_test <- Test Sets without processed data

X = dataset.iloc[:,2].values
y = dataset.iloc[:,4].values

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(444,) (50,) (444,) (50,)


In [719]:
vectorizer = CountVectorizer()

In [720]:
print(y_train.shape)
print(y_test.shape)

(444,)
(50,)


In [721]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [722]:
rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf.fit(X_train, y_train)
print("Training set score :{}".format(rf.score(X_train,y_train)))
print("Test set score :{}".format(rf.score(X_test,y_test)))

Training set score :0.8783783783783784
Test set score :0.56


In [723]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 0  0  6  2  0]
 [ 0  0  1  0  0]
 [ 0  0 23  5  0]
 [ 0  0  7  5  0]
 [ 0  0  0  1  0]]


In [724]:
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

attitudes and emotions       0.00      0.00      0.00         8
 hypothetical scenario       0.00      0.00      0.00         1
               request       0.62      0.82      0.71        28
            suggestion       0.38      0.42      0.40        12
              surprise       0.00      0.00      0.00         1

              accuracy                           0.56        50
             macro avg       0.20      0.25      0.22        50
          weighted avg       0.44      0.56      0.49        50



In [725]:
from sklearn.model_selection import cross_validate
avg_acc_score=0
for i, score in enumerate(cross_validate(rf, X_train, y_train, scoring='accuracy', cv=10)["test_score"]):
    avg_acc_score += score
    print(f"Accuracy for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Accuracy is: {avg_acc_score/10}")
print(f"\naverage 10-fold cross validation Accuracy percentage is: {(avg_acc_score/10)*100}")

Accuracy for the fold no. 0 on the test set: 0.5333333333333333
Accuracy for the fold no. 1 on the test set: 0.6222222222222222
Accuracy for the fold no. 2 on the test set: 0.5111111111111111
Accuracy for the fold no. 3 on the test set: 0.6444444444444445
Accuracy for the fold no. 4 on the test set: 0.5681818181818182
Accuracy for the fold no. 5 on the test set: 0.6590909090909091
Accuracy for the fold no. 6 on the test set: 0.6136363636363636
Accuracy for the fold no. 7 on the test set: 0.5454545454545454
Accuracy for the fold no. 8 on the test set: 0.5227272727272727
Accuracy for the fold no. 9 on the test set: 0.5909090909090909

average 10-fold cross validation Accuracy is: 0.581111111111111

average 10-fold cross validation Accuracy percentage is: 58.1111111111111


In [726]:
avg_precision=0
for i, score in enumerate(cross_validate(rf, X_train, y_train, scoring='precision_weighted', cv=10)["test_score"]):
    avg_precision += score
    print(f"Precision for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Precision is: {avg_precision/10}")
print(f"\naverage 10-fold cross validation Precision percentage is: {(avg_precision/10)*100}")

Precision for the fold no. 0 on the test set: 0.5043988269794721
Precision for the fold no. 1 on the test set: 0.576984126984127
Precision for the fold no. 2 on the test set: 0.4614695340501792
Precision for the fold no. 3 on the test set: 0.5814722911497104
Precision for the fold no. 4 on the test set: 0.5337465564738292
Precision for the fold no. 5 on the test set: 0.6493506493506493
Precision for the fold no. 6 on the test set: 0.599791013584117
Precision for the fold no. 7 on the test set: 0.5439882697947215
Precision for the fold no. 8 on the test set: 0.48770053475935826
Precision for the fold no. 9 on the test set: 0.5331744868035191

average 10-fold cross validation Precision is: 0.5472076289929684

average 10-fold cross validation Precision percentage is: 54.72076289929684


In [727]:
avg_recall=0
for i, score in enumerate(cross_validate(rf, X_train, y_train, scoring='recall_weighted', cv=10)["test_score"]):
    avg_recall += score
    print(f"Recall for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Recall is: {avg_recall/10}")
print(f"\naverage 10-fold cross validation Recall percentage is: {(avg_recall/10)*100} ")

Recall for the fold no. 0 on the test set: 0.5333333333333333
Recall for the fold no. 1 on the test set: 0.6222222222222222
Recall for the fold no. 2 on the test set: 0.5111111111111111
Recall for the fold no. 3 on the test set: 0.6444444444444445
Recall for the fold no. 4 on the test set: 0.5681818181818182
Recall for the fold no. 5 on the test set: 0.6590909090909091
Recall for the fold no. 6 on the test set: 0.6136363636363636
Recall for the fold no. 7 on the test set: 0.5454545454545454
Recall for the fold no. 8 on the test set: 0.5227272727272727
Recall for the fold no. 9 on the test set: 0.5909090909090909

average 10-fold cross validation Recall is: 0.581111111111111

average 10-fold cross validation Recall percentage is: 58.1111111111111 


In [728]:
from sklearn.model_selection import cross_validate
f1_score=0
for i, score in enumerate(cross_validate(rf, X_train, y_train, scoring='f1_weighted', cv=10)["test_score"]):
    f1_score += score
    print(f"f1-score for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation f1-score is: {f1_score/10}")
print(f"\naverage 10-fold cross validation f1-score percentage is: {(f1_score/10)*100}")

f1-score for the fold no. 0 on the test set: 0.510989010989011
f1-score for the fold no. 1 on the test set: 0.5983228511530398
f1-score for the fold no. 2 on the test set: 0.48059964726631393
f1-score for the fold no. 3 on the test set: 0.6071428571428572
f1-score for the fold no. 4 on the test set: 0.5338949843260188
f1-score for the fold no. 5 on the test set: 0.6516934046345811
f1-score for the fold no. 6 on the test set: 0.5988728252879197
f1-score for the fold no. 7 on the test set: 0.5274793388429753
f1-score for the fold no. 8 on the test set: 0.5044063079777366
f1-score for the fold no. 9 on the test set: 0.5536271808999081

average 10-fold cross validation f1-score is: 0.5567028408520363

average 10-fold cross validation f1-score percentage is: 55.670284085203626


## Improved model (Kernel SVM) - without preprocessing

In [729]:
svm_kernel = SVC(kernel = 'rbf', random_state = 0)
svm_kernel.fit(X_train, y_train)

y_pred_ksvm = svm_kernel.predict(X_test)
cm = confusion_matrix(y_test, y_pred_ksvm)
print(cm)
print(accuracy_score(y_test, y_pred_ksvm))
print(classification_report(y_test, y_pred_ksvm))
print("Training set score :{}".format(svm_kernel.score(X_train,y_train)))
print("Test set score :{}".format(svm_kernel.score(X_test,y_test)))

[[ 0  0  6  2  0]
 [ 0  0  1  0  0]
 [ 0  0 23  5  0]
 [ 0  0  6  6  0]
 [ 0  0  0  1  0]]
0.58
                        precision    recall  f1-score   support

attitudes and emotions       0.00      0.00      0.00         8
 hypothetical scenario       0.00      0.00      0.00         1
               request       0.64      0.82      0.72        28
            suggestion       0.43      0.50      0.46        12
              surprise       0.00      0.00      0.00         1

              accuracy                           0.58        50
             macro avg       0.21      0.26      0.24        50
          weighted avg       0.46      0.58      0.51        50

Training set score :0.7702702702702703
Test set score :0.58


In [730]:
from sklearn.model_selection import cross_validate
avg_acc_score=0
for i, score in enumerate(cross_validate(svm_kernel, X_train, y_train, scoring='accuracy', cv=10)["test_score"]):
    avg_acc_score += score
    print(f"Accuracy for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Accuracy is: {avg_acc_score/10}")
print(f"\naverage 10-fold cross validation Accuracy percentage is: {(avg_acc_score/10)*100}")

Accuracy for the fold no. 0 on the test set: 0.6444444444444445
Accuracy for the fold no. 1 on the test set: 0.6444444444444445
Accuracy for the fold no. 2 on the test set: 0.6
Accuracy for the fold no. 3 on the test set: 0.6222222222222222
Accuracy for the fold no. 4 on the test set: 0.5909090909090909
Accuracy for the fold no. 5 on the test set: 0.7272727272727273
Accuracy for the fold no. 6 on the test set: 0.7045454545454546
Accuracy for the fold no. 7 on the test set: 0.5681818181818182
Accuracy for the fold no. 8 on the test set: 0.5681818181818182
Accuracy for the fold no. 9 on the test set: 0.6590909090909091

average 10-fold cross validation Accuracy is: 0.632929292929293

average 10-fold cross validation Accuracy percentage is: 63.2929292929293


In [731]:
avg_precision=0
for i, score in enumerate(cross_validate(svm_kernel, X_train, y_train, scoring='precision_weighted', cv=10)["test_score"]):
    avg_precision += score
    print(f"Precision for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Precision is: {avg_precision/10}")
print(f"\naverage 10-fold cross validation Precision percentage is: {(avg_precision/10)*100}")

Precision for the fold no. 0 on the test set: 0.5757575757575757
Precision for the fold no. 1 on the test set: 0.5703703703703703
Precision for the fold no. 2 on the test set: 0.5306513409961686
Precision for the fold no. 3 on the test set: 0.5488991295442909
Precision for the fold no. 4 on the test set: 0.5303030303030304
Precision for the fold no. 5 on the test set: 0.6675324675324674
Precision for the fold no. 6 on the test set: 0.6565656565656566
Precision for the fold no. 7 on the test set: 0.5155791788856305
Precision for the fold no. 8 on the test set: 0.5016233766233765
Precision for the fold no. 9 on the test set: 0.6058884297520661

average 10-fold cross validation Precision is: 0.5703170556330633

average 10-fold cross validation Precision percentage is: 57.03170556330633


In [732]:
avg_recall=0
for i, score in enumerate(cross_validate(svm_kernel, X_train, y_train, scoring='recall_weighted', cv=10)["test_score"]):
    avg_recall += score
    print(f"Recall for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation Recall is: {avg_recall/10}")
print(f"\naverage 10-fold cross validation Recall percentage is: {(avg_recall/10)*100} ")

Recall for the fold no. 0 on the test set: 0.6444444444444445
Recall for the fold no. 1 on the test set: 0.6444444444444445
Recall for the fold no. 2 on the test set: 0.6
Recall for the fold no. 3 on the test set: 0.6222222222222222
Recall for the fold no. 4 on the test set: 0.5909090909090909
Recall for the fold no. 5 on the test set: 0.7272727272727273
Recall for the fold no. 6 on the test set: 0.7045454545454546
Recall for the fold no. 7 on the test set: 0.5681818181818182
Recall for the fold no. 8 on the test set: 0.5681818181818182
Recall for the fold no. 9 on the test set: 0.6590909090909091

average 10-fold cross validation Recall is: 0.632929292929293

average 10-fold cross validation Recall percentage is: 63.2929292929293 


In [733]:
from sklearn.model_selection import cross_validate
f1_score=0
for i, score in enumerate(cross_validate(svm_kernel, X_train, y_train, scoring='f1_weighted', cv=10)["test_score"]):
    f1_score += score
    print(f"f1-score for the fold no. {i} on the test set: {score}")
print(f"\naverage 10-fold cross validation f1-score is: {f1_score/10}")
print(f"\naverage 10-fold cross validation f1-score percentage is: {(f1_score/10)*100}")

f1-score for the fold no. 0 on the test set: 0.5998297147722434
f1-score for the fold no. 1 on the test set: 0.604040404040404
f1-score for the fold no. 2 on the test set: 0.5629895128103013
f1-score for the fold no. 3 on the test set: 0.5807334428024082
f1-score for the fold no. 4 on the test set: 0.5397727272727273
f1-score for the fold no. 5 on the test set: 0.6909090909090909
f1-score for the fold no. 6 on the test set: 0.679144385026738
f1-score for the fold no. 7 on the test set: 0.5337924701561065
f1-score for the fold no. 8 on the test set: 0.5325964358222423
f1-score for the fold no. 9 on the test set: 0.6188811188811189

average 10-fold cross validation f1-score is: 0.5942689302493382

average 10-fold cross validation f1-score percentage is: 59.426893024933825
